# Product Analytics Dashboard

This notebook contains an interactive dashboard for analyzing product data and recommendations.

In [ ]:
# Install required packages
!pip install dash jupyter-dash pandas plotly numpy

In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output

In [ ]:
# Load the data
df = pd.read_csv('../data/raw/ecommerce_data.csv')
df.head()

In [ ]:
# Create the Dash app
app = JupyterDash(__name__)

# Define the layout
app.layout = html.Div([
    # Header
    html.Div([
        html.H1('Product Analytics Dashboard',
                style={'textAlign': 'center', 'color': '#2c3e50', 'padding': '20px'})
    ]),
    
    # Main content
    html.Div([
        # Left column - Filters
        html.Div([
            html.H3('Filters', style={'color': '#34495e'}),
            html.Label('Select Category'),
            dcc.Dropdown(
                id='category-filter',
                options=[{'label': str(cat), 'value': cat} for cat in df['category'].unique()],
                multi=True,
                style={'marginBottom': '20px'}
            ),
        ], style={'width': '25%', 'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px'}),
        
        # Right column - Charts
        html.Div([
            # First row of charts
            html.Div([
                html.Div([
                    html.H4('Purchase Amount Distribution', style={'textAlign': 'center', 'color': '#34495e'}),
                    dcc.Graph(id='purchase-dist')
                ], style={'width': '48%', 'display': 'inline-block', 'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '10px'}),
                
                html.Div([
                    html.H4('Rating Distribution', style={'textAlign': 'center', 'color': '#34495e'}),
                    dcc.Graph(id='rating-dist')
                ], style={'width': '48%', 'display': 'inline-block', 'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '10px'}),
            ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '20px'}),
            
            # Second row of charts
            html.Div([
                html.Div([
                    html.H4('Category Distribution', style={'textAlign': 'center', 'color': '#34495e'}),
                    dcc.Graph(id='category-dist')
                ], style={'width': '48%', 'display': 'inline-block', 'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '10px'}),
                
                html.Div([
                    html.H4('Top Products', style={'textAlign': 'center', 'color': '#34495e'}),
                    dcc.Graph(id='top-products')
                ], style={'width': '48%', 'display': 'inline-block', 'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '10px'}),
            ], style={'display': 'flex', 'justifyContent': 'space-between'}),
        ], style={'width': '73%', 'padding': '20px'}),
    ], style={'display': 'flex', 'gap': '20px', 'padding': '20px', 'backgroundColor': '#ecf0f1'}),
])

# Callback for purchase amount distribution
@app.callback(
    Output('purchase-dist', 'figure'),
    [Input('category-filter', 'value')]
)
def update_purchase_dist(selected_categories):
    if selected_categories:
        filtered_df = df[df['category'].isin(selected_categories)]
    else:
        filtered_df = df
    
    fig = px.histogram(
        filtered_df,
        x='purchase_amount',
        nbins=30,
        title='Purchase Amount Distribution',
        color_discrete_sequence=['#3498db']
    )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=30, l=30, r=30, b=30)
    )
    return fig

# Callback for rating distribution
@app.callback(
    Output('rating-dist', 'figure'),
    [Input('category-filter', 'value')]
)
def update_rating_dist(selected_categories):
    if selected_categories:
        filtered_df = df[df['category'].isin(selected_categories)]
    else:
        filtered_df = df
    
    fig = px.box(
        filtered_df,
        y='rating',
        title='Rating Distribution',
        color_discrete_sequence=['#2ecc71']
    )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=30, l=30, r=30, b=30)
    )
    return fig

# Callback for category distribution
@app.callback(
    Output('category-dist', 'figure'),
    [Input('category-filter', 'value')]
)
def update_category_dist(selected_categories):
    if selected_categories:
        filtered_df = df[df['category'].isin(selected_categories)]
    else:
        filtered_df = df
    
    category_counts = filtered_df['category'].value_counts()
    
    fig = px.pie(
        values=category_counts.values,
        names=category_counts.index,
        title='Category Distribution',
        color_discrete_sequence=px.colors.qualitative.Set3
    )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=30, l=30, r=30, b=30)
    )
    return fig

# Callback for top products
@app.callback(
    Output('top-products', 'figure'),
    [Input('category-filter', 'value')]
)
def update_top_products(selected_categories):
    if selected_categories:
        filtered_df = df[df['category'].isin(selected_categories)]
    else:
        filtered_df = df
    
    product_stats = filtered_df.groupby('product_name').agg({
        'rating': 'mean',
        'purchase_amount': 'sum'
    }).sort_values('purchase_amount', ascending=True).tail(10)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=product_stats.index,
        x=product_stats['purchase_amount'],
        orientation='h',
        marker_color='#e74c3c'
    ))
    
    fig.update_layout(
        title='Top 10 Products by Purchase Amount',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=30, l=200, r=30, b=30),
        height=400
    )
    return fig

In [ ]:
# Run the dashboard
app.run_server(mode='inline')